### Import SQLlite and Pandas

In [43]:
import sqlite3, pandas

In [44]:
db = 'chinook.db'

### Define helper function to connect to DB.
This function takes a SQL query as an argument and returns a Pandas dataframe

In [45]:
def run_query(q):
    with sqlite3.connect(db) as conn:
        return pandas.read_sql(q, conn)

### Define helper function to run SQL commands using the SQLite 3 module

In [46]:
def run_command(c):
      with sqlite3.connect(db) as conn:
        conn.isolation_level = None
        conn.execute(c)

### Define helper function to view current state of DB

In [47]:
def show_tables():
    query = '''
        SELECT
        name, type
        FROM sqlite_master
        WHERE type IN ("table","view");
    '''
    return run_query(query)
show_tables()

,name,type
0,album,table
1,artist,table
2,customer,table
3,employee,table
4,genre,table
5,invoice,table
6,invoice_line,table
7,media_type,table
8,playlist,table
9,playlist_track,table


### Tasks

#### Get genre by tracks sold (just in US) by absolute num and % of total

In [48]:
q = '''
    SELECT *
    FROM track
    INNER JOIN genre ON genre.genre_id = track.genre_id
    LIMIT 10;
'''
run_query(q)

,track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price,genre_id,name
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99,1,Rock
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99,1,Rock
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99,1,Rock
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99,1,Rock
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99,1,Rock
5,6,Put The Finger On You,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.99,1,Rock
6,7,Let's Get It Up,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",233926,7636561,0.99,1,Rock
7,8,Inject The Venom,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,0.99,1,Rock
8,9,Snowballed,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",203102,6599424,0.99,1,Rock
9,10,Evil Walks,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",263497,8611245,0.99,1,Rock


#### Query joins the invoice-line and invoice table to track table to view the number of times each track (from USA) has been sold.

In [55]:
q = '''
WITH usa_tracks_sold AS
   (
    SELECT track.genre_id genre, CAST(COUNT(invoice_line.track_id) AS FLOAT) quantity_sold
    FROM invoice_line
    INNER JOIN invoice ON invoice.invoice_id = invoice_line.invoice_id
    INNER JOIN track ON track.track_id = invoice_line.track_id
    WHERE invoice.billing_country = 'USA'
    GROUP BY track.genre_id
    ORDER BY 2 DESC
    LIMIT 10
   )
SELECT  genre.name, 
        quantity_sold, 
        ROUND(
                usa_tracks_sold.quantity_sold / 
                (
                    SELECT SUM (usa_tracks_sold.quantity_sold) from usa_tracks_sold
                ),
            2) percentage_sold
FROM usa_tracks_sold
INNER JOIN genre ON genre.genre_id = usa_tracks_sold.genre;
'''
run_query(q)


,name,quantity_sold,percentage_sold
0,Rock,561.0,0.55
1,Alternative & Punk,130.0,0.13
2,Metal,124.0,0.12
3,R&B/Soul,53.0,0.05
4,Blues,36.0,0.04
5,Alternative,35.0,0.03
6,Latin,22.0,0.02
7,Pop,22.0,0.02
8,Hip Hop/Rap,20.0,0.02
9,Jazz,14.0,0.01


In [50]:
genre_sales_usa = run_query(q)

#### The four best-selling genres for in the USA are:
1. Rock
2. Alternarive & Punk
3. Metal
4. R&B/Soul

#### Determine the best-performing sales agents.
- Determine the sales number by employee...

In [89]:
q = '''
WITH sales_by_support_rep AS
   (
    SELECT 
        customer.support_rep_id,
        SUM(CAST(invoice.total AS FLOAT)) sales
    FROM customer
    INNER JOIN invoice ON invoice.customer_id = customer.customer_id
    GROUP BY 1
    ORDER BY sales DESC
   )
SELECT 
    employee.first_name || " " || employee.last_name name,
    sales
FROM sales_by_support_rep
INNER JOIN employee ON
    employee_id = sales_by_support_rep.support_rep_id
'''
run_query(q)

,name,sales
0,Jane Peacock,1731.51
1,Margaret Park,1584.00
2,Steve Johnson,1393.92


#### These are the three highest performing sales reps

#### Analyze the sales data by country

In [90]:
c = '''
CREATE VIEW top_5_names AS
     SELECT
         first_name,
         count(customer_id) count
     FROM customer
     GROUP by 1
     ORDER by 2 DESC
     LIMIT 5;
'''
run_command(c)

q = 'SELECT * FROM top_5_names;'
run_query(q)

,first_name,count
0,Frank,2
1,Mark,2
2,Aaron,1
3,Alexandre,1
4,Astrid,1


#### Analyzing data from different countries.
1. First subquery 'country_or_other checks that for a given order, the country in the invoice table corresponds to the country value in the customer table. If there isnt a valid value then we provide 'Other' instead.
2. Second subquery 'aggregated_country_stats' calculates aggregate stats and groups the results by country. We also add a caculated numeric column based on the country or 'Other' value to sort the results for clearer analysis.
3. Main query just selects the results from the subquery

In [95]:
q = '''
WITH 
    country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
     ),
     aggregated_country_stats AS
     (
     SELECT
        country,
        count(distinct customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / count(distinct customer_id) customer_lifetime_value,
        SUM(unit_price) / count(distinct invoice_id) average_order,
    CASE
        WHEN country = "Other" THEN 1
        ELSE 0
    END AS sort
    FROM 
        country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    )
SELECT * FROM aggregated_country_stats;
'''
run_query(q)

,country,customers,total_sales,customer_lifetime_value,average_order,sort
0,USA,13,1040.49,80.037692,7.942672,0
1,Canada,8,535.59,66.948750,7.047237,0
2,Brazil,5,427.68,85.536000,7.011148,0
3,France,5,389.07,77.814000,7.781400,0
4,Germany,4,334.62,83.655000,8.161463,0
5,Czech Republic,2,273.24,136.620000,9.108000,0
6,United Kingdom,3,245.52,81.840000,8.768571,0
7,Portugal,2,185.13,92.565000,6.383793,0
8,India,2,183.15,91.575000,8.721429,0
9,Other,15,1094.94,72.996000,7.448571,1


#### Get % of tracks purchases made as part of a whole album vs not
1) First subquery gets the first track purchase (using) MIN function per invoice. It doesn't matter which track we get from the invoice...

In [97]:
 q = '''
 WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;
'''
run_query(q)

,album_purchase,number_of_invoices,percent
0,no,500,0.814332
1,yes,114,0.185668
